In [1]:
# Eigen value and Eigen vector problem

# code developer: Hyunju Lee and Kyungtaek Jun

# First eigen value and eigen vector
#  1   0 *  c1 = 1 *  c1
# -4   3    2c1       2c1

# Second eigen value and eigen vector
#  1   0 *  0    =  3 *  0
# -4   3    3c2          3c2

# Find second solution
# x = a + 2b : Eigenvector
#     c + 2d 
# r = e + 2f  : Eigenvalue
# We only use integer valuse of x_i, eigenvalues, and eigenvectors between [-3, 3]
# We test this algorithm with D-Wave qpu solver for HUBO model.
# We will convert the HUBO cdoe to the QUBO code which can be used in the D-Wave Quantum annealer directly.

import numpy as np
import random, math
import copy
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod

A = np.array([[1, 0], [-4, 3]])
print("Matrix A:")
print(A)

Matrix A:
[[ 1  0]
 [-4  3]]


In [2]:
def Q_mat(qubits,QM):
    max_d = format(len(str(qubits)), '02')
    
    # linear terms
    Q = {}
    for i in range(qubits): 
        linear_term = format(i + 1, max_d)
        exec("Q.update({('q%s','q%s'):%s})"%(linear_term, linear_term, format(QM[i][i])))

    # quadratic terms
    for i in range(qubits-1):
        for j in range(i+1,qubits):
            if QM[i][j] != 0:
                qdrt1 = format(i + 1, max_d)
                qdrt2 = format(j + 1, max_d)
                exec("Q.update({('q%s','q%s'):%s})"%(qdrt1,qdrt2,format(QM[i][j])))
    return Q
              

In [3]:
A.shape

(2, 2)

In [4]:
# Test code for integer x and ramda
# x1 = q1 + 2q2 - q3 - 2q4
# x2 = q5 + 2q6 - q7 - 2q8
# x3 = q9 + 2q10 - q11 - 2q12
# x4 = q13 + 2q14 - q15 - 2q16
# ramda = q17 + 2q18 - q19 - 2q20

(Dimension, Dimension_same) = A.shape
qubit_eigen = 2
qubits = 2
QM = np.zeros((2*qubits*(Dimension+1), 2*qubits*(Dimension+1)))
              
### Linear terms in Eq. 9 ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef = pow(2,2*l)*pow(A[k][i],2)
            po1 = 2*qubits*i + l    
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef    # q+
            QM[po2][po2] = QM[po2][po2] + cef    # q-

### First quadratic terms in Eq. 9 ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef            
                        
 ### Second quadratic terms in Eq. 10 ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef           

### Third quadratic terms in Eq. 12 ###
for k in range(Dimension):
    for l in range(qubits):
        for l1 in range(qubits):
            qcef = pow(2, l+2*l1+1)*A[k][k]
            po1 = 2*qubits*k + l1
            po2 = 2*qubits*Dimension + qubits + l
            QM[po1][po2] = QM[po1][po2] + qcef
            po3 = 2*qubits*k + qubits + l1
            po4 = 2*qubits*Dimension + qubits + l
            QM[po3][po4] = QM[po3][po4] + qcef
            po5 = 2*qubits*k + l1
            po6 = 2*qubits*Dimension + l
            QM[po5][po6] = QM[po5][po6] - qcef
            po7 = 2*qubits*k + qubits + l1
            po8 = 2*qubits*Dimension + l
            QM[po7][po8] = QM[po7][po8] - qcef

### Final quadratic terms in Eq. 13 ###              
for k in range(Dimension):
    for l1 in range(qubits):
        for l2 in range(qubits):
            qcef = pow(2, 2*l1+2*l2)
            po1 = 2*qubits*k + l2
            po2 = 2*qubits*Dimension + l1
            QM[po1][po2] = QM[po1][po2] + qcef
            po3 = 2*qubits*k + qubits + l2
            po4 = 2*qubits*Dimension + l1
            QM[po3][po4] = QM[po3][po4] + qcef
            po5 = 2*qubits*k + l2
            po6 = 2*qubits*Dimension + qubits + l1
            QM[po5][po6] = QM[po5][po6] + qcef
            po7 = 2*qubits*k + qubits + l2
            po8 = 2*qubits*Dimension + qubits + l1
            QM[po7][po8] = QM[po7][po8] + qcef             
Q = Q_mat(2*qubits*(Dimension+1),QM)

In [5]:
max_d = format(len(str(2*qubits*(Dimension+1))), '02')

### Two cubic terms: 2nd row in Eq. 12 and 2nd row in Eq. 13 ###              
for k in range(Dimension):
    for l1 in range(qubits):
        for l2 in range(qubits-1):
            for l3 in range(l2+1,qubits):
                ccef1 = pow(2, l1+l2+l3+2)*A[k][k]
                ccef2 = pow(2, 2*l1+l2+l3+1)
                # q(k,l2, +) q(k,l3, +) q(l1, +)
                po11 = 2*qubits*k + l2
                po12 = 2*qubits*k + l3
                po13 = 2*qubits*(Dimension) + l1
                cval1 = -ccef1 + ccef2
                # q(k,l2, +) q(k,l3, +) q(l1, -)
                po21 = 2*qubits*k + l2
                po22 = 2*qubits*k + l3
                po23 = 2*qubits*(Dimension) + qubits + l1
                cval2 = ccef1 + ccef2
                # q(k,l2, -) q(k,l3, -) q(l1, +)
                po31 = 2*qubits*k + qubits + l2
                po32 = 2*qubits*k + qubits + l3
                po33 = 2*qubits*(Dimension) + l1
                cval3 = -ccef1 + ccef2
                # q(k,l2, -) q(k,l3, -) q(l1, -)
                po41 = 2*qubits*k + qubits + l2
                po42 = 2*qubits*k + qubits + l3
                po43 = 2*qubits*(Dimension) + qubits + l1
                cval4 = ccef1 + ccef2
                ### print 4 values q[poi1]q[poi2]q[poi3] = cvali
            
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po11+1,max_d),format(po12+1,max_d), format(po13+1,max_d), format(cval1)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po21+1,max_d),format(po22+1,max_d), format(po23+1,max_d), format(cval2)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po31+1,max_d),format(po32+1,max_d), format(po33+1,max_d), format(cval3)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po41+1,max_d),format(po42+1,max_d), format(po43+1,max_d), format(cval4)))
       
                ### Final cubic terms: 3rd row in Eq. 13 ###                    
                # q(k,l1, +) q(l2, +) q(l3, +)
                po11 = 2*qubits*k + l1
                po12 = 2*qubits*(Dimension) + l2
                po13 = 2*qubits*(Dimension) + l3
                cval1 = ccef2              
                # q(k,l1, +) q(l2, -) q(l3, -)
                po21 = 2*qubits*k + l1
                po22 = 2*qubits*(Dimension) + qubits + l2
                po23 = 2*qubits*(Dimension) + qubits + l3
                cval2 = ccef2  
                # q(k,l1, -) q(l2, +) q(l3, +)
                po31 = 2*qubits*k + qubits + l1
                po32 = 2*qubits*(Dimension) + l2
                po33 = 2*qubits*(Dimension) + l3
                cval3 = ccef2                
                # q(k,l1, -) q(l2, -) q(l3, -)
                po41 = 2*qubits*k + qubits + l1
                po42 = 2*qubits*(Dimension) + qubits + l2
                po43 = 2*qubits*(Dimension) + qubits + l3
                cval4 = ccef2                
                ### print 4 values q[poi1]q[poi2]q[poi3] = cvali
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po11+1,max_d),format(po12+1,max_d), format(po13+1,max_d), format(cval1)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po21+1,max_d),format(po22+1,max_d), format(po23+1,max_d), format(cval2)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po31+1,max_d),format(po32+1,max_d), format(po33+1,max_d), format(cval3)))
                exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po41+1,max_d),format(po42+1,max_d), format(po43+1,max_d), format(cval4)))

### Final cubic terms: 3rd and 4th rows in Eq. 12 ###                    
# We have to add here        
for k in range(Dimension-1):
    for i in range(k+1,Dimension):
        for l in range(qubits):
            for l1 in range(qubits):          
                for l2 in range(qubits):   
                    qtval = pow(2, l+l1+l2+1)*(A[k][i]+A[i][k])
                    # q(k,l2,-) q(i,l1,+)  q(l,+)
                    po11 = 2*qubits*k + qubits + l2
                    po12 = 2*qubits*i + l1
                    po13 = 2*qubits*Dimension + l
                    # q(k,l2,+) q(i,l1,-)  q(l,+)
                    po21 = 2*qubits*k + l2
                    po22 = 2*qubits*i + qubits + l1
                    po23 = 2*qubits*Dimension + l
                    # q(k,l2,+) q(i,l1,+)  q(l,+) -
                    po31 = 2*qubits*k + l2
                    po32 = 2*qubits*i + l1
                    po33 = 2*qubits*Dimension + l
                    # q(k,l2,-) q(i,l1,-)  q(l,+) -
                    po41 = 2*qubits*k + qubits + l2
                    po42 = 2*qubits*i + qubits + l1
                    po43 = 2*qubits*Dimension + l
                    # q(k,l2,-) q(i,l1,+) q(l,-) -
                    po51 = 2*qubits*k + qubits + l2
                    po52 = 2*qubits*i + l1
                    po53 = 2*qubits*Dimension + qubits + l
                    # q(k,l2,+) q(i,l1,-)  q(l,-) -
                    po61 = 2*qubits*k + l2
                    po62 = 2*qubits*i + qubits + l1
                    po63 = 2*qubits*Dimension + qubits + l
                    # q(k,l2,+) q(i,l1,+)  q(l,-)
                    po71 = 2*qubits*k + l2
                    po72 = 2*qubits*i + l1
                    po73 = 2*qubits*Dimension + qubits + l
                    # q(k,l2,-) q(i,l1,-)  q(l,-)
                    po81 = 2*qubits*k + qubits + l2 
                    po82 = 2*qubits*i + qubits + l1
                    po83 = 2*qubits*Dimension + qubits + l
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po11+1,max_d),format(po12+1,max_d), format(po13+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po21+1,max_d),format(po22+1,max_d), format(po23+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po31+1,max_d),format(po32+1,max_d), format(po33+1,max_d), format(-qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po41+1,max_d),format(po42+1,max_d), format(po43+1,max_d), format(-qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po51+1,max_d),format(po52+1,max_d), format(po53+1,max_d), format(-qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po61+1,max_d),format(po62+1,max_d), format(po63+1,max_d), format(-qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po71+1,max_d),format(po72+1,max_d), format(po73+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s'):%s})"%(format(po81+1,max_d),format(po82+1,max_d), format(po83+1,max_d), format(qtval)))
                    


                    
### First Quartic terms in Eq. 12 ###
for k in range(Dimension):
    for l1 in range(qubits-1):
        for l2 in range(l1+1,qubits):
            for l3 in range(qubits-1):          
                for l4 in range(l3+1,qubits):
                    qtval = pow(2, l1+l2+l3+l4+2)
                    # q(k,l3, +) q(k,l4, +) q(l1, +) q(l2, +)
                    po11 = 2*qubits*k + l3
                    po12 = 2*qubits*k + l4
                    po13 = 2*qubits*(Dimension) + l1
                    po14 = 2*qubits*(Dimension) + l2
                    # q(k,l3, +) q(k,l4, +) q(l1, -) q(l2, -)
                    po21 = 2*qubits*k + l3
                    po22 = 2*qubits*k + l4
                    po23 = 2*qubits*(Dimension) + qubits + l1
                    po24 = 2*qubits*(Dimension) + qubits + l2              
                    # q(k,l3, -) q(k,l4, -) q(l1, +) q(l2, +)
                    po31 = 2*qubits*k + qubits + l3
                    po32 = 2*qubits*k + qubits + l4
                    po33 = 2*qubits*(Dimension) + l1
                    po34 = 2*qubits*(Dimension) + l2
                    # q(k,l3, -) q(k,l4, -) q(l1, -) q(l2, -)
                    po41 = 2*qubits*k + qubits + l3
                    po42 = 2*qubits*k + qubits + l4
                    po43 = 2*qubits*(Dimension) + qubits + l1
                    po44 = 2*qubits*(Dimension) + qubits + l2
                    ### print 4 values q[poi1]q[poi2]q[poi3] = cvali
                    exec("Q.update({('q%s','q%s','q%s','q%s'):%s})"%(format(po11+1,max_d),format(po12+1,max_d), format(po13+1,max_d), format(po14+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s','q%s'):%s})"%(format(po21+1,max_d),format(po22+1,max_d), format(po23+1,max_d), format(po24+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s','q%s'):%s})"%(format(po31+1,max_d),format(po32+1,max_d), format(po33+1,max_d), format(po34+1,max_d), format(qtval)))
                    exec("Q.update({('q%s','q%s','q%s','q%s'):%s})"%(format(po41+1,max_d),format(po42+1,max_d), format(po43+1,max_d), format(po44+1,max_d), format(qtval)))

In [6]:
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
sampleset = dimod.ExactPolySolver().sample_hubo(Q)
#sampleset.first.energy
print(sampleset)

     q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc.
0      0   0   0   0   0   0   0   0   0   0   0   0    0.0       1
7      0   0   0   0   0   0   0   0   0   0   0   1    0.0       1
120    0   0   0   0   0   0   0   0   0   1   0   1    0.0       1
127    0   0   0   0   0   0   0   0   0   1   0   0    0.0       1
896    0   0   0   0   0   0   0   0   0   1   1   0    0.0       1
903    0   0   0   0   0   0   0   0   0   1   1   1    0.0       1
1016   0   0   0   0   0   0   0   0   0   0   1   1    0.0       1
1023   0   0   0   0   0   0   0   0   0   0   1   0    0.0       1
1024   0   0   0   0   0   0   0   0   1   0   1   0    0.0       1
1031   0   0   0   0   0   0   0   0   1   0   1   1    0.0       1
1144   0   0   0   0   0   0   0   0   1   1   1   1    0.0       1
1151   0   0   0   0   0   0   0   0   1   1   1   0    0.0       1
1632   0   0   0   0   1   0   0   1   1   1   0   0    0.0       1
1633   0   0   0   0   1   0   1   1   1   1   0

In [7]:
# lambda = 1 : x2=2*x1 /  lambda = 3 : x1=0, x2 =t(integer -3:1:3) 

# energy = 0
energies = sampleset.record.energy
energy0_nums = np.where(energies==0)[0]
for idx in range(len(energy0_nums)):
    sol1 = sampleset.record[energy0_nums[idx]][0]
    x1 = sol1[0]+2*sol1[1]-sol1[2]-2*sol1[3]
    x2 = sol1[4]+2*sol1[5]-sol1[6]-2*sol1[7]
    lambda1 = sol1[8]+2*sol1[9]-sol1[10]-2*sol1[11]
    # except for cases satisfying (x1,x2) = (0,0)
    if x1!=0 or x2!=0:
        print('(order,x1,x2,lambda) = ',(energy0_nums[idx],x1,x2,lambda1))

(order,x1,x2,lambda) =  (1632, 0, -1, 3)
(order,x1,x2,lambda) =  (1633, 0, -2, 3)
(order,x1,x2,lambda) =  (1663, 0, 1, 3)
(order,x1,x2,lambda) =  (1921, 0, -1, 3)
(order,x1,x2,lambda) =  (1950, 0, -3, 3)
(order,x1,x2,lambda) =  (1951, 0, -2, 3)
(order,x1,x2,lambda) =  (2019, -1, -2, 1)
(order,x1,x2,lambda) =  (2063, 1, 2, 1)
(order,x1,x2,lambda) =  (2145, 0, -1, 3)
(order,x1,x2,lambda) =  (2174, 0, 1, 3)
(order,x1,x2,lambda) =  (2175, 0, 2, 3)
(order,x1,x2,lambda) =  (2432, 0, 3, 3)
(order,x1,x2,lambda) =  (2433, 0, 2, 3)
(order,x1,x2,lambda) =  (2463, 0, 1, 3)
